Solutions for windows notebook

In [ ]:
# Load the extension
%load_ext sql

In [ ]:
# Connect to DuckDB
# NOTE: ONLY 1 NOTEBOOK CAN BE CONNECTED TO DUCKDB AT ANY TIME
%sql duckdb:///../../tpch.db

In [ ]:
%%sql
-- USER EXERCISE 1
SELECT
  o_orderdate,
  o_custkey,
  o_totalprice,
  o_totalprice - lag (o_totalprice) OVER (
    PARTITION BY
      o_custkey
    ORDER BY
      o_orderdate
  ) AS totalprice_diff
FROM
  orders
ORDER BY
  o_custkey,
  o_orderdate
LIMIT
  50;

In [ ]:
%%sql
-- USER EXERCISE 2
SELECT
  order_month,
  o_custkey,
  total_price,
  ROUND(
    AVG(total_price) OVER (
      PARTITION BY
        o_custkey
      ORDER BY
        CAST(order_month AS DATE) RANGE BETWEEN INTERVAL '1' MONTH PRECEDING
        AND INTERVAL '1' MONTH FOLLOWING
    ),
    2
  ) AS consecutive_three_mo_total_price_avg,
  ROUND(
    AVG(total_price) OVER (
      PARTITION BY
        o_custkey
      ORDER BY
        order_month ROWS BETWEEN 1 PRECEDING
        AND 1 FOLLOWING
    ),
    2
  ) AS three_mo_total_price_avg
FROM
  (
    SELECT
      strftime (o_orderdate, '%Y-%m-01') AS order_month,
      o_custkey,
      SUM(o_totalprice) AS total_price
    FROM
      orders
    GROUP BY
      1,
      2
  )
ORDER BY
  o_custkey,
  order_month
LIMIT
  50;